## Cosine Similarity between Z-Scores

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import sys

# reference main directory in existing folder
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import pandas as pd 
import sys
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from src.paths import RAW_DATA_DIR, CLEANED_DATA_DIR, QUADRANT_DATA_DIR, MISMATCHES
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# show all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# read in the data
fangraph = pd.read_csv(CLEANED_DATA_DIR / 'fangraphs_zscores_merged.csv')

In [27]:
df_clean = fangraph.copy()
# Standardize features
df_clean = df_clean.dropna(axis=1, thresh = 2800)
df_clean = df_clean.dropna(axis=0)

# Move all string columns next to each other
string_columns = df_clean.select_dtypes(include=['object']).columns
other_columns = [col for col in df_clean.columns if col not in string_columns]

# Reorder the columns by first placing string columns and then other columns
df_clean = df_clean[string_columns.tolist() + other_columns]

In [28]:
df_clean.head()

,Name,NameASCII,Throws,Team,Role,player_name,classified_role,id,Classification,PlayerId,pitcher,Season,Age,G,GS,IP,TBF,W,L,CG,ShO,SV,BS,HLD,SD,MD,Pulls,ERA,R,ER,H,HR,SO,BB,IBB,HBP,WP,BK,Events,GB,LD,FB,IFFB,BU,IFH,BUH,Balls,Strikes,Pitches,RS,RS_per_9,K_pct,BB_pct,K_minus_BB_pct,K_per_9,BB_per_9,K_to_BB,H_per_9,HR_per_9,AVG,WHIP,BABIP,LOB_pct,GB_pct,LD_pct,FB_pct,IFFB_pct,GB_to_FB,HR_to_FB,IFH_pct,BUH_pct,FIP,xFIP,SIERA,tERA,kwERA,E_minus_F,RAR,WAR,Dollars,WPA,RE24,REW,pLI,inLI,gmLI,exLI,WPA_to_LI,Clutch,OSwing_pct,ZSwing_pct,Swing_pct,OContact_pct,ZContact_pct,Contact_pct,Zone_pct,FStrike_pct,SwStr_pct,CStr_pct,CSW_pct,ERA_minus,FIP_minus,xFIP_minus,Pace,RA9_WAR,BIP_Wins,LOB_Wins,FDP_Wins,Pull_pct,Cent_pct,Oppo_pct,Soft_pct,Med_pct,Hard_pct,TTO_pct,FRM,K_pct_plus,BB_pct_plus,K_per_9_plus,BB_per_9_plus,K_to_BB_plus,H_per_9_plus,HR_per_9_plus,AVG_plus,WHIP_plus,BABIP_plus,LOB_pct_plus,GB_pct_plus,LD_pct_plus,FB_pct_plus,HR_to_FB_pct_plus,Pull_pct_plus,Cent_pct_plus,Oppo_pct_plus,Med_pct_plus,Hard_pct_plus,EV,LA,Barrels,Barrel_pct,maxEV,HardHit,HardHit_pct,FA_pct,FAv,wFB,wFB_per_c,FA_pct_sc,vFA_sc,FA_X_sc,FA_Z_sc,wFA_sc,wFA_per_c_sc,OSwing_pct_sc,ZSwing_pct_sc,Swing_pct_sc,OContact_pct_sc,ZContact_pct_sc,Contact_pct_sc,Zone_pct_sc,botERA,botOvr_FA,botStf_FA,botCmd_FA,botOvr,botStf,botCmd,botxRV100,Stf_plus_FA,Loc_plus_FA,Pit_plus_FA,Stuff_plus,Location_plus,Pitching_plus,average_batters_faced_per_appearance_SP,average_innings_pitched_per_appearance_SP,average_batters_faced_per_appearance_RP,average_innings_pitched_per_appearance_RP,z_total_score
0,NeftalA­ Feliz,Neftali Feliz,R,- - -,RP,Neftali Feliz,Setup Pitcher,NeftalA­ Feliz2021,average_rp,18,491703,2021,33,5,0,4.0,19,0,1,0,0,0,1,0,0,2,4,9.000002,4,4,5,1,3,1,0,1,0,0,14,5,4,5,1,0,0,0,20,47,67,0,0.000000,0.1579,0.0526,0.1053,6.750002,2.250001,3.0000,11.250003,2.250001,0.294118,1.500000,0.307692,0.5357,0.357143,0.285714,0.357143,0.200000,1.000000,0.200000,0.000000,0.000000,6.420031,5.375481,4.519107,7.421088,4.889752,2.579971,-1.885527,-0.174990,-1.399920,-0.875182,-4.499,-0.420531,1.072752,0.609160,1.061280,0.263950,-0.137091,-0.678738,0.5000,0.7619,0.5821,0.7391,0.9375,0.8205,0.3134,0.6842,0.1045,0.1194,0.2239,213.324493,148.609662,126.817213,25.5870,-0.395077,-0.001708,-0.218379,-0.220087,0.4286,0.4286,0.1429,0.2143,0.5000,0.2857,0.2632,0.000000,67.733131,59.330599,75.367393,66.017851,76.108133,136.696882,183.262120,123.424821,115.998542,107.010757,74.114072,82.038249,137.175994,100.214061,145.942786,107.747591,121.632996,57.166409,97.430746,88.985942,84.847499,15.615007,3,0.2143,108.634,4,0.2857,0.7015,95.0851,-0.57793,-1.229638,0.7015,94.989364,-7.174255,9.762127,-0.594629,-1.265168,0.3750,0.7714,0.5821,0.8333,0.8148,0.8205,0.5224,5.299472,37.391871,49.884001,34.194912,41.998539,46.695011,43.207210,0.690299,106.865957,90.140426,90.834043,104.316418,95.123881,95.873134,0.0,0.0,3.800000,1.000000,-9.810740
1,Tommy Hunter,Tommy Hunter,R,NYM,RP,Tommy Hunter,Middle Reliever,Tommy Hunter2021,elite_rp,1157,488984,2021,34,3,0,6.0,26,0,0,0,0,0,0,0,1,0,3,0.000000,0,0,4,0,5,3,0,1,1,0,17,7,5,4,1,1,0,0,34,58,92,4,6.000000,0.1923,0.1154,0.0769,7.500000,4.500000,1.6667,6.000000,0.000000,0.181818,1.166667,0.235294,1.0000,0.437500,0.312500,0.250000,0.250000,1.750000,0.000000,0.000000,0.000000,3.503364,4.679604,4.692691,4.712296,5.229833,-3.503364,0.566930,0.057611,0.460886,0.200408,2.882,0.303443,0.715007,0.599233,0.602467,0.720867,0.141754,0.138534,0.3269,0.6750,0.4783,0.7647,0.9630,0.8864,0.4348,0.6923,0.0543,0.1413,0.1957,0.000000,83.446871,110.270416,22.4127,0.276844,0.077289,0.141945,0.219233,0.5294,0.1765,0.2941,0.1765,0.5882,0.2353,0.3077,0.045000,82.495480,130.070928,83.741526,132.035668,63.423444,72.904985,0.000000,76.298980,90.221066,81.831755,138.346268,100.496855,150.036244,70.149843,0.000000,133.099965,50.084175,117.695548,114.624408,73.282540,87.655624,10.628437,1,0.0588,106.499,3,0.1765,0.4130,92.3947,1.43093,3.765605,0.0652,93.116669,-4.543333,9.463333,0.487117,8.118621,0.28

### Use 2023 Season 

In [29]:
# use 2023 data 
df_2023 = df_clean[df_clean['Season'] == 2023]
df_2023.shape

(838, 179)

In [30]:
df_2023.head()

,Name,NameASCII,Throws,Team,Role,player_name,classified_role,id,Classification,PlayerId,pitcher,Season,Age,G,GS,IP,TBF,W,L,CG,ShO,SV,BS,HLD,SD,MD,Pulls,ERA,R,ER,H,HR,SO,BB,IBB,HBP,WP,BK,Events,GB,LD,FB,IFFB,BU,IFH,BUH,Balls,Strikes,Pitches,RS,RS_per_9,K_pct,BB_pct,K_minus_BB_pct,K_per_9,BB_per_9,K_to_BB,H_per_9,HR_per_9,AVG,WHIP,BABIP,LOB_pct,GB_pct,LD_pct,FB_pct,IFFB_pct,GB_to_FB,HR_to_FB,IFH_pct,BUH_pct,FIP,xFIP,SIERA,tERA,kwERA,E_minus_F,RAR,WAR,Dollars,WPA,RE24,REW,pLI,inLI,gmLI,exLI,WPA_to_LI,Clutch,OSwing_pct,ZSwing_pct,Swing_pct,OContact_pct,ZContact_pct,Contact_pct,Zone_pct,FStrike_pct,SwStr_pct,CStr_pct,CSW_pct,ERA_minus,FIP_minus,xFIP_minus,Pace,RA9_WAR,BIP_Wins,LOB_Wins,FDP_Wins,Pull_pct,Cent_pct,Oppo_pct,Soft_pct,Med_pct,Hard_pct,TTO_pct,FRM,K_pct_plus,BB_pct_plus,K_per_9_plus,BB_per_9_plus,K_to_BB_plus,H_per_9_plus,HR_per_9_plus,AVG_plus,WHIP_plus,BABIP_plus,LOB_pct_plus,GB_pct_plus,LD_pct_plus,FB_pct_plus,HR_to_FB_pct_plus,Pull_pct_plus,Cent_pct_plus,Oppo_pct_plus,Med_pct_plus,Hard_pct_plus,EV,LA,Barrels,Barrel_pct,maxEV,HardHit,HardHit_pct,FA_pct,FAv,wFB,wFB_per_c,FA_pct_sc,vFA_sc,FA_X_sc,FA_Z_sc,wFA_sc,wFA_per_c_sc,OSwing_pct_sc,ZSwing_pct_sc,Swing_pct_sc,OContact_pct_sc,ZContact_pct_sc,Contact_pct_sc,Zone_pct_sc,botERA,botOvr_FA,botStf_FA,botCmd_FA,botOvr,botStf,botCmd,botxRV100,Stf_plus_FA,Loc_plus_FA,Pit_plus_FA,Stuff_plus,Location_plus,Pitching_plus,average_batters_faced_per_appearance_SP,average_innings_pitched_per_appearance_SP,average_batters_faced_per_appearance_RP,average_innings_pitched_per_appearance_RP,z_total_score
988,Tommy Hunter,Tommy Hunter,R,NYM,RP,Tommy Hunter,Middle Reliever,Tommy Hunter2023,average_rp,1157,488984,2023,36,14,0,23.2,106,0,1,0,0,0,0,1,1,4,8,6.845061,20,18,28,6,20,5,0,2,0,0,79,37,10,32,1,0,3,0,117,251,368,16,6.084498,0.1887,0.0472,0.1415,7.605623,1.901406,4.0000,10.647872,2.281687,0.282828,1.394364,0.301370,0.5639,0.468354,0.126582,0.405063,0.031250,1.156250,0.187500,0.081081,0.0,5.747994,4.687740,4.022362,4.642953,4.520617,1.097067,-3.003590,-0.290334,-2.322670,-0.430858,-9.096,-0.914462,0.435751,0.354823,0.61150,0.710050,-0.350132,-0.638639,0.3302,0.7613,0.5123,0.8143,0.8475,0.8351,0.4223,0.6887,0.0842,0.1685,0.2527,164.123831,133.063646,107.048763,18.3870,-0.630813,-0.039543,-0.300936,-0.340479,0.5063,0.2785,0.2152,0.1519,0.4937,0.3544,0.2925,0.350000,84.752426,54.504666,88.604172,56.981715,155.496201,123.139491,185.923207,113.764758,104.718163,101.423524,78.544000,108.510070,62.397915,110.820182,144.808888,123.038761,80.191494,89.212667,95.580963,108.000094,88.597088,6.432798,10,0.1266,110.654,34,0.4304,0.4687,92.1279,3.456690,2.009704,0.0790,92.386205,-4.106448,9.664759,0.129710,0.447274,0.2982,0.7041,0.5150,0.7647,0.8551,0.8307,0.5341,4.325810,40.012945,34.095981,37.348669,52.090204,39.113682,59.946747,-0.003035,78.348148,112.925926,109.192593,82.804360,105.339510,100.527793,7.000000,2.000000,8.000000,2.000000,-11.123835
989,Matt Bush,Matt Bush,R,MIL,RP,Matt Bush,Setup Pitcher,Matt Bush2023,suboptimal_rp,1246,456713,2023,37,12,0,10.1,48,0,2,0,0,1,3,1,4,3,8,9.580676,11,11,11,5,10,6,0,0,0,0,32,9,11,12,1,0,1,0,72,122,194,1,0.870971,0.2083,0.1250,0.0833,8.709706,5.225823,1.6667,9.580676,4.354853,0.261905,1.645167,0.222222,0.6000,0.281250,0.343750,0.375000,0.083333,0.750000,0.416667,0.111111,0.0,9.351834,4.981520,4.854827,11.940876,5.218730,0.228842,-7.399680,-0.706893,-5.655150,-1.210920,-4.032,-0.395128,1.537240,1.398570,1.70520,1.535600,-0.721416,-0.066307,0.2960,0.6957,0.4381,0.5946,0.7917,0.7059,0.3557,0.5208,0.1289,0.1856,0.3144,218.540152,212.202027,113.750962,19.5137,-0.699669,0.151387,-0.144162,0.007225,0.5000,0.3125,0.1875,0.2188,0.4063,0.3750,0.4375,0.160000,93.580804,144.437365,101.466544,156.608545,64.790084,110.797688,354.855096,105.348488,123.553680,74.787043,83.570816,65.161030,169.449339,102.595247,321.797528,121.500776,89.987615,77.733096,78.655168,114.267957,89.473434,15.887281,6,0.1875,112.223,13,0.4063,0.5876,94.4211,-3.015150,-2.644868,0.5206,94.443562,-3

In [31]:
# Filter for only the players who fall under suboptimal and average categories under the Classification column, average_rp, suboptimal_rp, average_sp, suboptimal_sp, average_rp, suboptimal_rp, average_sp, suboptimal_sp
suboptimal_players = df_2023[df_2023['Classification'].isin(['average_rp', 'suboptimal_rp', 'average_sp', 'suboptimal_sp'])]
elite_and_strong_players = df_2023[df_2023['Classification'].isin(['elite_rp', 'strong_rp', 'elite_sp', 'strong_sp'])]

# Include only the columns that are relevant to the analysis
string_columns = ['Name', 'Team', 'classified_role', 'Classification']
numeric_columns = df_2023.iloc[:, 13:].columns.tolist()

df_2023 = df_2023[string_columns + numeric_columns]
suboptimal_players = suboptimal_players[string_columns + numeric_columns]
elite_and_strong_players = elite_and_strong_players[string_columns + numeric_columns]

In [34]:
# Filtering for elite and strong players
is_elite_strong = df_2023['Classification'].isin(['elite_rp', 'strong_rp', 'elite_sp', 'strong_sp'])
elite_and_strong_players = df_2023[is_elite_strong]

is_suboptimal_average = df_2023['Classification'].isin(['average_rp', 'suboptimal_rp', 'average_sp', 'suboptimal_sp'])
suboptimal_players = df_2023[is_suboptimal_average]

# Group by 'classified_role' and calculate mean of non-PCA numeric columns only
elite_and_strong_mean = elite_and_strong_players.groupby('classified_role')[numeric_columns].mean()
suboptimal_mean = suboptimal_players.groupby('classified_role')[numeric_columns].mean()


In [ ]:
# Assuming df_2023 is your original dataframe

# Filter for suboptimal and elite/strong players
suboptimal_players = df_2023[df_2023['Classification'].isin(['average_rp', 'suboptimal_rp', 'average_sp', 'suboptimal_sp'])]
elite_and_strong_players = df_2023[df_2023['Classification'].isin(['elite_rp', 'strong_rp', 'elite_sp', 'strong_sp'])]

# Include only relevant columns
string_columns = ['Name', 'Team', 'classified_role', 'Classification']
non_pca_numeric_columns = [col for col in df_2023.columns if 'PCA' not in col and df_2023[col].dtype.kind in 'biufc']

# Calculate the mean statistics for elite and strong players
elite_mean = elite_and_strong_players.groupby('classified_role')[non_pca_numeric_columns].mean()

In [36]:
import pandas as pd
from scipy.spatial.distance import cosine

# Define the similarity function as provided...
def calculate_similarity_to_roles(player_stats, role_means):
    """
    Calculate the similarity of a player's statistics to the mean statistics of successful players in each role.

    Parameters:
    - player_stats (pd.Series): A Pandas Series containing the player's statistics.
    - role_means (pd.DataFrame): A Pandas DataFrame where the index represents the role and the columns are the mean statistics for that role.

    Returns:
    - similarities (dict): A dictionary with roles as keys and similarity scores as values.
    """
    similarities = {}
    for role, mean_stats in role_means.iterrows():
        # Calculate the cosine similarity (or any other similarity measure)
        similarity = 1 - cosine(player_stats.values, mean_stats.values)
        similarities[role] = similarity
    return similarities

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_25180\3507197286.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  suggested_changes = pd.concat([suggested_changes, pd.DataFrame(rows_to_add)], ignore_index=True)


In [ ]:
# Initialize suggested_changes dataframe
suggested_changes = pd.DataFrame(columns=['Name', 'Current Role', 'Suggested Role', 'Similarity Score'])

rows_to_add = []
for index, player in suboptimal_players.iterrows():
    player_stats = player[non_pca_numeric_columns]
    similarities = calculate_similarity_to_roles(player_stats, elite_mean)

    suggested_role = max(similarities, key=similarities.get)
    actual_role = player['classified_role']
    max_similarity_score = similarities[suggested_role]

    if suggested_role != actual_role:
        rows_to_add.append({
            'Name': player['Name'],
            'Current Role': actual_role,
            'Suggested Role': suggested_role,
            'Similarity Score': max_similarity_score
        })

suggested_changes = pd.concat([suggested_changes, pd.DataFrame(rows_to_add)], ignore_index=True)

In [39]:
# Sort the suggested_changes dataframe by 'Similarity Score' in descending order
suggested_changes.sort_values(by='Similarity Score', ascending=False, inplace=True)
suggested_changes.reset_index(drop=True, inplace=True)
suggested_changes.head(10)

,Name,Current Role,Suggested Role,Similarity Score
0,Taj Bradley,Setup Pitcher,Starting Pitcher,0.998389
1,Brandon Pfaadt,Long Reliever,Starting Pitcher,0.998328
2,Tony Gonsolin,Long Reliever,Starting Pitcher,0.997992
3,Seth Martinez,Setup Pitcher,Long Reliever,0.997742
4,Chris Murphy,Middle Reliever,Long Reliever,0.996845
5,Joan Adon,Starting Pitcher,Long Reliever,0.996782
6,Luis Severino,Setup Pitcher,Starting Pitcher,0.996618
7,Tucker Davidson,Starting Pitcher,Long Reliever,0.996537
8,GA©nesis Cabrera,Setup Pitcher,Closer,0.996477
9,Seranthony DomA­nguez,Middle Reliever,Closer,0.996403
